## Importamos las librerías necesarias

In [1]:
from math import exp
import random
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from scipy.special import expit
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold

## Código modelo de factorización matricial de Bernoulli

In [ ]:
def sigmoid(x):
    try:
        return 1 / (1 + exp(-x))
    except:
        # print(f"Cannot calculate the sigmoid function for x={x}. Rounding to {1 if x > 0 else 0}")
        return 1 if x > 0 else 0


def calculate_gradient(prev_value, is_one_hot: bool, dot: float, item):
    return prev_value + (1 - sigmoid(dot)) * item if is_one_hot else prev_value - sigmoid(dot) * item


def update_factor(element, gradient, learning_rate: float, regularisation: float):
    return element + learning_rate * (gradient - regularisation * element)


class BeMF:
    num_factors = num_iters = 0
    learning_rate = regularisation = 0.0
    possible_scores = []  # eg. 1,2,3,4,5
    U = [[[]]]  # User-factor matrices for each score
    V = [[[]]]  # Item-factor matrices for each score
    user_ids = []  # All the different users
    item_ids = []  # All the different items
    number_of_users = 0
    number_of_items = 0
    ratings = [[]] # The matrix for each user-item combination with the score if the user rated it and None if not
    __cached_MAE = -1 # Caches the MAE result. Reset upon .fit() calls
    predictions_matrix = [[]] # The prediction given for each user-item combination

    def __init__(self, possible_scores: [], user_item_rating_matrix: [[]], user_ids: [], item_ids: [], num_factors: int, num_iters: int, learning_rate: int, regularisation: int, seed: int, verbose=True):
        self.num_factors = num_factors
        self.num_iters = num_iters
        self.learning_rate = learning_rate
        self.regularisation = regularisation
        self.possible_scores = possible_scores
        self.user_ids = user_ids
        self.number_of_users = len(user_ids)
        self.item_ids = item_ids
        self.number_of_items = len(item_ids)
        self.ratings = user_item_rating_matrix
        random.seed(seed)

        self.U = [[[random.random() for _ in range(num_factors)] for _ in user_ids] for _ in possible_scores]
        self.V = [[[random.random() for _ in range(num_factors)] for _ in item_ids] for _ in possible_scores]

        if verbose:
            print("*BeMF model setup completed*")
            self.print_status()

    def print_status(self):
        print(f"num_factors:\t{self.num_factors}")
        print(f"num_iters:\t{self.num_iters}")
        print(f"learning_rate:\t{self.learning_rate}")
        print(f"regularisation:\t{self.regularisation}")
        print(f"possible_scores:\t{self.possible_scores}")
        print(f"user_ids:\t{len(self.user_ids)}")
        print(f"item_ids:\t{len(self.item_ids)}")
        print(f"ratings:\t({len(self.ratings)}, {len(self.ratings[0])})")
        print(f"U:\t({len(self.U)}, {len(self.U[0])}, {len(self.U[0][0])})")
        print(f"V:\t({len(self.V)}, {len(self.V[0])}, {len(self.V[0][0])})")


    def fit(self, verbose=False, make_predictions_matrix=False):
        for i in range(1, self.num_iters+1):
            self.__cached_MAE = -1
            for s in range(len(self.possible_scores)):
                score = self.possible_scores[s]
                for user_index in range(self.number_of_users):
                    self.__update_users_factors(user_index, self.U[s], self.V[s], score)
                for item_index in range(self.number_of_items):
                    self.__update_items_factors(item_index, self.U[s], self.V[s], score)
            if verbose:
                self.__print_current_iteration(i)
        if make_predictions_matrix:
            self.make_predictions_matrix()
        if verbose:
            print("Training concluded")


    def __print_current_iteration(self, i: int):
        if i == 1:
            print("Starting fitting process. Please wait.")
            return
        if (i % 10) == 0:
            print(f"\t{i} iterations - MAE: {self.evaluate_MAE()}")
            return
        print(".", end="")

    
    def evaluate_MAE(self):
        """Calculates the Mean Absolute Error (MAE) of the model. The value should get closer to 0 as the training advances.

        Returns:
            float: The result of the calculations
        """
        if self.__cached_MAE < 0:
            pred_df = pd.DataFrame(self.make_predictions_matrix()).fillna(0)
            real_df = pd.DataFrame(self.ratings).fillna(0)
            self.__cached_MAE = mean_absolute_error(real_df, pred_df)
        return self.__cached_MAE


    def make_predictions_matrix(self):
        self.predictions_matrix = np.array([np.array([self.predict(user_index, item_index, False)
                                            for item_index in range(self.number_of_items)])
                                            for user_index in range(self.number_of_users)])
        return self.predictions_matrix



    def __update_users_factors(self, user_index: int, U: [[]], V: [[]], score: int):
        gradients = [0] * self.num_factors

        for item_index in range(len(V)):
            if not self.ratings[user_index][item_index]:
                continue  # Not rated, skip
            is_one_hot = self.ratings[user_index][item_index] == score
            dot_product = np.dot(U[user_index], V[item_index])
            gradients = [calculate_gradient(gradients[k], is_one_hot, dot_product, V[item_index][k]) for k in range(self.num_factors)]
        
        U[user_index] = [update_factor(U[user_index][k], gradients[k], self.learning_rate, self.regularisation) for k in range(self.num_factors)]


    def __update_items_factors(self, item_index: int, U: [[]], V: [[]], score: int):
        gradients = [0] * self.num_factors

        for user_index in range(len(U)):
            if not self.ratings[user_index][item_index]:
                continue  # Not rated, skip
            is_one_hot = self.ratings[user_index][item_index] == score
            dot_product = np.dot(U[user_index], V[item_index])
            gradients = [calculate_gradient(gradients[k], is_one_hot, dot_product, U[user_index][k]) for k in range(self.num_factors)]

        V[item_index] = [update_factor(V[item_index][k], gradients[k], self.learning_rate, self.regularisation) for k in range(self.num_factors)]


    def get_probability(self, user_index: int, item_index: int, score_index):
        """Calculate the probability of the user rating the item with the given score_index

        Args:

            `score_index` (int): Index of the score present in `possible_scores`

        Returns:

            float: The calculated probability
        """
        if score_index >= len(self.possible_scores):
            return f"Error: index {score_index} out of range {len(self.possible_scores)}"
        dot_product = sigmoid(np.dot(self.U[score_index][user_index], self.V[score_index][item_index]))
        sum = 0.0

        for i in range(len(self.possible_scores)):
            sum += sigmoid(np.dot(self.U[i][user_index], self.V[i][item_index]))

        try:
            return dot_product/sum
        except ZeroDivisionError:
            return 0


    def predict(self, user_index: int, item_index: int, use_cached_results: bool = True):
        """
        Args:

            `use_cached_results` (bool): If False forces recalculation of values

        Returns:

            int: the score most likely to be given by the user at `user_index` to the item at `item_index`
        """
        if user_index >= len(self.U[0]):
            return f"Error: index {user_index} out of range {len(self.U[0])}"
        if item_index >= len(self.V[0]):
            return f"Error: index {item_index} out of range {len(self.V[0])}"
        if use_cached_results:
            return self.predictions_matrix[user_index][item_index]

        maximum = self.get_probability(user_index, item_index, 0)
        index = 0

        for r in range(1, len(self.possible_scores)):
            probability = self.get_probability(user_index, item_index, r)
            if (maximum < probability):
                maximum = probability
                index = r
        
        return self.possible_scores[index]


    def predict_proba(self, user_index: int, item_index: int):
        prediction = self.predict(user_index, item_index)
        return self.get_probability(user_index, item_index, self.possible_scores.index(prediction))

# Importamos los datos 

In [2]:
df_datos = pd.read_excel('C:/Users/amidonga/Documents/TFG/Datos_Sinteticos_CON_TRAMITES.xlsx')

### Corrección de las variables

In [ ]:
# Cambiamos 0 por 1 y 1 por 2
for columna in ['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']:
    df_datos[columna] = df_datos[columna] +1

### Calculamos los n usuarios más parecidos a un usuario concreto

In [ ]:
def calcular_usuarios_similares(df, usuario_concreto, n):
    # Filtrar el dataframe por las características relevantes para el cálculo de similitud
    features = ['Edad', 'Procedencia', 'Sexo', 'Situación de dependencia', 'Sector económico',
                'Renta anual neta', 'Estado civil', 'Número de hijos']
    df_filt = df[features].copy()

    # Codificar las variables categóricas utilizando one-hot encoding
    df_encoded = pd.get_dummies(df_filt)
    # Obtener el índice del usuario concreto
    idx_usuario_concreto = df_filt.index[df_filt.index == usuario_concreto]
    
    # Calcular la similitud del coseno entre el usuario concreto y todos los demás usuarios
    similarities = cosine_similarity(df_encoded.iloc[idx_usuario_concreto], df_encoded)

    # Obtener los índices de los usuarios más similares (excluyendo al usuario concreto)
    similar_users_indices = np.argsort(similarities[0])[::-1][:n]
    
    # Agregar el índice del usuario concreto al conjunto de usuarios similares
    #similar_users_indices = np.concatenate((idx_usuario_concreto, similar_users_indices))

    # Obtener los datos de los usuarios más similares
    similar_users = df.loc[similar_users_indices]

    return similar_users


### Función para preparar la matriz de valoraciones a partir del dataframe de datos

In [ ]:
def crear_ratings_matrix(df):    
    # Obtén el número de filas y columnas
    num_rows, num_cols = df.shape
    # Crea una matriz de tamaño adecuado, inicializada con None
    ratings_matrix = np.full((num_rows, num_cols),None, dtype=object)

    # Itera sobre las filas del dataframe
    for row_idx, row in df.iterrows():
        # Itera sobre las columnas del dataframe
        for col_idx, col_label in enumerate(df.columns):
            # Obtiene el valor de la celda
            value = row[col_label]
            # Si el valor no es NaN, colócalo en la matriz
            if not pd.isnull(value):
                ratings_matrix[row_idx, col_idx] = int(value)
        
    return ratings_matrix


# Ejemplo de uso concreto

In [ ]:
### BIEN la valoración es 1 ->0


usuario= 3 
item = 2  #empieza en 0 para predict 1=0
# Calculamos los 200 usuarios más similares al usuario elegido.
usuarios_similares = calcular_usuarios_similares(df_datos, usuario_concreto = usuario, n=200)

# Creamos la matriz de valoraciones de estos usuarios
df_ratings_matrix = usuarios_similares[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
df_ratings_matrix.reset_index(inplace=True)
df_ratings_matrix=df_ratings_matrix[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
ratings_matrix_u=crear_ratings_matrix(df_ratings_matrix)


# Ajustamos el modelo
possible_scores = [1,2]
user_ids = list(range(0,200))  # Lista de ID de usuarios
item_ids = [1,2,3,4,5,6,7]  # Lista de ID de elementos
num_factors = 10
num_iters = 200
learning_rate = 0.02
regularisation = 0.1
seed = 42

bemf_model = BeMF(possible_scores, ratings_matrix_u, user_ids, item_ids, num_factors, num_iters, learning_rate, regularisation, seed)
    
# Actualizaciones del proceso
bemf_model.fit(verbose=True)

# predicción 
prediction = bemf_model.predict(0, item-1)

probabilidad = round(bemf_model.get_probability(0, item-1,prediction-1),4)

print('La predicción para el usuario',usuario,'y el item',item,'es la valoración',prediction-1,'con probabilidad',probabilidad)

# Función para automatizar la predicción 

In [ ]:
def prediccion(usuario,item, df_para_usuarios_similares): # usuarios de 0 a n (indices tabla).  Item 0 se corresponde con el 1
    # Tomamos los 200 usuarios más similares a él
    usuarios_similares = calcular_usuarios_similares(df_para_usuarios_similares, usuario_concreto = usuario, n=350)
    df_ratings_matrix = usuarios_similares[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
    df_ratings_matrix.reset_index(inplace=True)
    df_ratings_matrix=df_ratings_matrix[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
    
    # Creamos la matriz asociada a este dataframe
    ratings_matrix_u=crear_ratings_matrix(df_ratings_matrix)
    
    # Ajustamos el modelo
    possible_scores = [1,2]
    user_ids = list(range(0,350))  # Lista de ID de usuarios
    item_ids = [1,2,3,4,5,6,7]  # Lista de ID de elementos
    num_factors = 10
    num_iters = 100
    learning_rate = 0.02 
    regularisation = 0.1
    seed = 42

    bemf_model = BeMF(possible_scores, ratings_matrix_u, user_ids, item_ids, num_factors, num_iters, learning_rate, regularisation, seed)
    
    # Actualizaciones del proceso
    bemf_model.fit(verbose=True)
    
    # predicción 
    prediction = bemf_model.predict(0, item -1)
    
    return prediction

## Estuadiamos el rendimiento del modelo

### Accuracy True/False

Calculamos la mtriz de valoraciones general

In [ ]:
ratings_matrix_general = crear_ratings_matrix(df_datos[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']])

Estudiamos el redimiento calculando la predicción sobre las valoraciones de las que ya tenemos datos

## Automatizamos la evaluación del modelo

In [ ]:
def evaluar_modelo(ratings_matrix_general,df_para_usuarios_similares, n_usuarios):
    accuracies = []
    confusion_matrix = np.zeros((2, 2), dtype=int)
    TP= 0
    FP= 0
    FN = 0
    
    for u in range(n_usuarios):
        for i in range(7):
            valor_real = ratings_matrix_general[u][i]
            if valor_real != None:
                prediction =prediccion(u,i+1,df_para_usuarios_similares)
                print('valor',valor_real)
                print('prediccion',prediction)
                accuracies.append(prediction==valor_real)
                confusion_matrix[valor_real - 1, prediction - 1] += 1
                
                if (valor_real == 2) &(prediction == 2):
                    TP +=1
                elif (valor_real == 1) &(prediction == 2):
                    FP +=1
                elif(valor_real == 2) &(prediction == 1):
                    FN +=1
                    
    confusion_matrix_percent = np.round(confusion_matrix.astype(float) / confusion_matrix.sum(axis=1, keepdims=True) * 100, 2)
    acc = accuracies.count(True)*100/len(accuracies)
    
    # F1 score
    F1 = TP/(TP+0.5*(FP+FN))
    
    # Configurar el formato de impresión
    np.set_printoptions(formatter={'float': lambda x: "{:.2f}".format(x)})

    return acc,confusion_matrix_percent,F1

# Evaluación del sistema sobre un muestreo aleatorio
Para seleccionar un subconjunto aleatorio, utilizamos la función sample() de pandas.El parámetro "n" indica el número de muestras aleatorias que deseas seleccionar

In [ ]:
df_datos_subset = df_datos.sample(n=100).reset_index()

Evaluamos el modelo sobre el subconjunto de datos seleccionado. Utilizamos aún así el conjunto de datos completo para calcular los usuarios similares de cada usuario.

In [ ]:
ratings_matrix_general_subset = crear_ratings_matrix(df_datos_subset[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']])

In [ ]:
evaluar_modelo(ratings_matrix_general=ratings_matrix_general_subset,df_para_usuarios_similares = df_datos, n_usuarios=100)

# Evaluación sobre los usuarios más activos
Calculamos el número de evaluaciones que ha realizado cada usuario. Tomaremos solo aquellos usuarios que hayan valorado más de tres items para llevar a cabo la evaluación de nuestro modelo.

In [ ]:
# Añadimos una variable que indica el número de evaluaciones realizadas por cada usuario
df_datos['num_evaluations'] = df_datos[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']].notnull().sum(axis=1)

# Tomamos solo aquellos usuarios con más de tres valoraciones.
df_datos_mas3_valoraciones =df_datos[df_datos['num_evaluations']>3][['Comunidad Autónoma', 'Edad', 'Procedencia', 'Sexo',
       'Situación de dependencia', 'Sector económico', 'Renta anual neta',
       'Estado civil', 'Número de hijos', 'T.1.', 'T.2.', 'T.3.', 'T.4.',
       'T.5.', 'T.6.', 'T.7.']].reset_index()

# Tomamos un subconjunto de los usuarios más activos
df_activos_subset = df_datos_mas3_valoraciones.sample(n=100).reset_index()


Evaluamos el modelo sobre el subconjunto de datos seleccionado. Utilizamos el conjunto de usuarios más activos para calcular los usuarios similares.

In [ ]:
ratings_matrix_activos_subset = crear_ratings_matrix(df_activos_subset[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']])

In [ ]:
evaluar_modelo(ratings_matrix_general=ratings_matrix_activos_subset,df_para_usuarios_similares = df_datos_mas3_valoraciones, n_usuarios=100)